# Clize
## A Python approach to command-line argument parsing
Yann Kaiser - [@YannKsr](https://twitter.com/YannKsr)

## Hello world!

In [115]:
def greet(name=None, *, no_capitalize=False):
    """Returns a greeting for a given person or the whole world."""
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

In [116]:
greet()

'Hello world!'

In [117]:
greet("john")

'Hello John!'

In [118]:
greet("dave", no_capitalize=True)

'Hello dave!'

next: Python 2

In [119]:
from sigtools import modifiers

@modifiers.kwoargs('no_capitalize')
def greet(name=None, no_capitalize=False):
    """Returns a greeting for a given person or the whole world."""
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

but further examples Py3

next: argparse answer

In [120]:
greet("dave", no_capitalize=True)

'Hello dave!'

## This is how you write a command-line interface for it
(according to the docs)

In [121]:
import argparse

def greetcli(args=None):
    parser = argparse.ArgumentParser(prog='greeter')
    parser.add_argument("name", nargs='?')
    parser.add_argument("--no-capitalize", action="store_const", const=True)

    args = parser.parse_args(args=args)

    print(greet(args.name, no_capitalize=args.no_capitalize))

greetcli()

usage: greeter [-h] [--no-capitalize] [name]
greeter: error: unrecognized arguments: -f --profile-dir /home/epsy/.ipython/profile_default


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


    $ python greet.py john 

In [122]:
greetcli(['john'])

Hello John!


    $ python greet.py dave --no-capitalize

In [123]:
greetcli(['dave', '--no-capitalize'])

Hello dave!


What did ``greetcli`` do?

### What did ``greetcli`` do?

Positional argument ➡ Positional argument

Named argument ➡ Named argument

This is redundant!

## Meet Clize

In [124]:
from clize import run

run(greet)

/home/epsy/.local/lib/python3.4/site-packages/IPython/kernel/__main__.py: Unknown option '-f'
Usage: /home/epsy/.local/lib/python3.4/site-packages/IPython/kernel/__main__.py [OPTIONS] [name]


SystemExit: 2

To exit: use 'exit', 'quit', or Ctrl-D.


In [125]:
from clize import run as orig_run

def rrun(*args, **kwargs):
    orig_run(*args, exit=False, **kwargs)

def run(*args, **kwargs): pass

    $ python greeter.py

In [126]:
rrun(greet, args=['greeter.py'])

Hello world!


    $ python greeter.py john

In [127]:
rrun(greet, args=['greeter.py', 'john'])

Hello John!


    $ python greeter.py dave --no-capitalize

In [128]:
rrun(greet, args=['greeter.py', 'dave', '--no-capitalize'])

Hello dave!


In [129]:
def greet(name=None, *, no_capitalize=False):
    """Returns a greeting for a given person or the whole world.
    
    name: The name of the person to greet
    
    no_capitalize: Don't capitalize the given name
    """
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

    $ python greet.py --help

In [130]:
rrun(greet, args=['greet.py', '--help'])

Usage: greet.py [OPTIONS] [name]

Returns a greeting for a given person or the whole world.

Arguments:
  name              The name of the person to greet

Options:
  --no-capitalize   Don't capitalize the given name

Other actions:
  -h, --help        Show the help


## More simple things

### Want an int? ➞ Let the default be an int!

In [131]:
def greet(name, *, repeat=1):
    """Returns a greeting for a given person.
    
    name: The name of the person to greet
    
    repeat: Greet the person this many times
    """
    message = "Hello {}!".format(name.title())
    return '\n'.join(message for _ in range(repeat))

    $ python greet.py john --repeat 3

In [132]:
rrun(greet, args=['greet.py', 'john', '--repeat', '3'])

Hello John!
Hello John!
Hello John!


### Collect all positional ➞ Use ``*args``

In [133]:
def greet(*name):
    """Returns a greeting for a given person.
    
    name: The name of the person to greet
    
    repeat: Greet the person this many times
    """
    return "Hello {}!".format(' '.join(name).title())

    $ python greet.py john doe

In [134]:
rrun(greet, args=['greet.py', 'john', 'doe'])

Hello John Doe!


### Want an alias for your option? ➞ Use an annotation

In [135]:
def greet(name=None, *, no_capitalize:'C'=False):
    """Returns a greeting for a given person or the whole world.
    
    name: The name of the person to greet
    
    no_capitalize: Don't capitalize the given name
    """
    if name is None:
        return "Hello world!"
    else:
        if not no_capitalize:
            name = name.title()
        return "Hello {}!".format(name)

    $ python greet.py dave -C

In [136]:
rrun(greet, args=['greet.py', 'dave', '-C'])

Hello dave!


In [137]:
# Py2

@modifiers.annotate(no_capitalize='C')
@modifiers.kwoargs('no_capitalize')
def greet(name=None, no_capitalize=False):
    ...

Also, many other uses of annotations, ex: adding parameters to parameters

## I heard you like parameters...

In [138]:
from clize.parameters import argument_decorator

@argument_decorator
def name_argument(val, *, no_capitalize:'C'=False):
    """For each name:
    
    no_capitalize: Don't capitalize this name"""
    if not no_capitalize:
        return val.title()
    return val

In [139]:
def greet(*names: name_argument):
    """Greets some people.
    
    names: The people to address"""
    if not names:
        return 'Hello world!'
    greetings = []
    for name in names:
        greetings .append('Hello {}!'.format(name))
    return '\n'.join(greetings)

run(greet)

    $ python greet.py john -C jane

In [140]:
rrun(greet, args=['greet.py', 'john', '-C', 'jane'])

Hello John!
Hello jane!


## Run multiple functions

In [141]:
def part(name):
    """Wishes someone goodbye.
    
    name: The person to address"""
    return "Goodbye {}".format(name)

In [142]:
run(greet, part)

    $ python greet.py --help

In [143]:
rrun(greet, part, args=['greet.py', '--help'])

Usage: greet.py command [args...]

Commands:
  greet   Greets some people.
  part    Wishes someone goodbye.


Share features?

### Share features between commands

In [151]:
from sigtools.wrappers import wrapper_decorator

@wrapper_decorator
def message_func(wrapped, *args, repeat:'r'=1, **kwargs):
    """
    repeat: Amount of times the message is repeated
    """
    message = wrapped(*args, **kwargs)
    return '\n'.join(message for _ in range(repeat))

@message_func
def greet(*names: name_argument):
    ...

@message_func
def part(*names: name_argument):
    ...

* ``wrapper_decorator`` makes it so that Clize can know ``*args`` and ``**kwargs`` refer to the decorated function


    $ python greet.py greet -C -r 3 dave

In [153]:
@message_func
def greet(*names: name_argument):
    """Greets some people.
    
    names: The people to address"""
    if not names:
        return 'Hello world!'
    greetings = []
    for name in names:
        greetings .append('Hello {}!'.format(name))
    return '\n'.join(greetings)

@message_func
def part(*names: name_argument):
    """Wishes some people goodbye.
    
    names: The people to address"""
    if not names:
        return 'Goodbye!'
    greetings = []
    for name in names:
        greetings .append('Goodbye {}!'.format(name))
    return '\n'.join(greetings)

In [154]:
rrun(greet, part, args=['greet.py', 'greet', '-r', '2', 'john', '-C', 'dave'])

Hello John!
Hello dave!
Hello John!
Hello dave!


# ``pip install --user clize==3.0b1``

https://clize.readthedocs.org/

[Clize community on Google+](https://plus.google.com/communities/101146333300650079362)

### Yann Kaiser

[@YannKsr](https://twitter.com/YannKsr)

[kaiser.yann@gmail.com](mailto:kaiser.yann@gmail.com)

FAQ page  